In [ ]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd

In [ ]:
import sys
sys.path.append("../sv_system/")

In [ ]:
import torch
from system.sv_test import sv_test  

sv_embeds = np.load("../sv_system/embeddings/voices_fbank64_voxc12_embeds/sv_embeds.npy") 
sv_embeds = torch.from_numpy(sv_embeds) 

trial = pd.read_csv("../sv_system/datasets/voices/voices_dev_trial.csv")
eer, thres, scores = sv_test(sv_embeds, trial)
trial = trial.assign(score=scores.numpy())

In [ ]:
trial.groupby('label').score.hist(alpha=0.8)

In [ ]:
fp_trials = trial[(trial.score > thres) & (trial.label == 0)] 
fn_trials = trial[(trial.score < thres) & (trial.label == 1)]

In [ ]:
def get_info(id_str, q_key):
    keys = ["room", "noise", "spk_id", "ch_id", "sg_id", "mc_id", "mc_t", "mc_l", "mc_deg"]
    if q_key not in keys:
        print("valid key:{}".format(keys))
        raise KeyError
    id_splits = id_str.split('-')
    info = {}
    for i, key in enumerate(keys):
        info[key] = id_splits[i+3] # first three are useless
    
    return info[q_key]

In [ ]:
for key in ['ch_id', 'sg_id', 'noise', 'mc_id', 'mc_l', 'mc_deg']:
    fn_trials['comp_'+key] = fn_trials.apply(lambda x: (get_info(x.enroll_id, key), get_info(x.test_id, key)), axis=1)

In [ ]:
fn_trials.sort_values('score').head(20)

In [ ]:
fn_trials.comp_mc_l.value_counts()

In [ ]:
fn_trials.comp_mc_id.value_counts()

In [ ]:
fn_trials.comp_mc_deg.value_counts()

In [ ]:
for key in ['ch_id', 'sg_id', 'mc_id', 'mc_l', 'mc_deg']:
    fp_trials['comp_'+key] = fp_trials.apply(lambda x: (get_info(x.enroll_id, key), get_info(x.test_id, key)), axis=1)

In [ ]:
fp_trials.sort_values('score', ascending=False)[]

In [ ]:
trial.head()

### Unique configurations 

In [ ]:
trial.enroll_id.apply(lambda x: get_info(x, 'noise')).unique()

In [ ]:
trial.test_id.apply(lambda x: get_info(x, 'noise')).unique()